In [ ]:
import socket
import threading
from tkinter import *
from tkinter import font
from tkinter import ttk
import urllib.request
from PIL import ImageTk,Image
from tkinter import filedialog
import os
PORT = 6000
SERVER = socket.gethostbyname(socket.gethostname()) 
FORMAT = "utf-8"
client = socket.socket(socket.AF_INET,socket.SOCK_STREAM) 
client.connect((SERVER, PORT)) 

class GUI: 

    def __init__(self): 

        self.Window = Tk() 
        self.Window.withdraw() 
        
        self.login = Toplevel() 
        self.login.title("CHATROOM") 
        self.login.resizable(width = True, height = True) 
        self.login.configure(width = 800, height = 600,bg="#111111") 
        
        self.yourName = Label(self.login, text = "Please enter the nome of \n room to continue", 
                    justify = CENTER, font = "Helvetica 14",bg="#222222",fg="#999999") 
        self.yourName.place(relheight = 0.15, relwidth = 0.4, relx = 0.55, rely = 0.62) 

        self.URL = Label(self.login, text = "Please enter Link URL",
                         justify = CENTER, font = "Helvetica 14",bg="#222222",fg="#999999") 
        
        self.URL.place(relheight = 0.15, relwidth = 0.4, relx = 0.55, rely = 0.37) 
        
        self.entryName = Entry(self.login, font = "Helvetica 14") 
        
        self.entryName.place(relwidth = 0.35, relheight = 0.05, relx = 0.52, rely = 0.75) 
        
        self.entryName.focus() 
        
#         self.labelNameURL = Label(self.login, text = "Name: ", font = "Helvetica 12") 

#         self.labelNameURL.place(relheight = 0.2, relx = 0.1, rely = 0.48) 

        self.entryNameURL = Entry(self.login, font = "Helvetica 14") 
        
        self.entryNameURL.place(relwidth = 0.35, relheight = 0.05, relx = 0.52, rely = 0.48)
        
        self.name=""
        def goAhead(self,name):
            self.name=name
        self.go = Button(self.login, text = "Join", font = "Helvetica 14 bold", command = lambda: self.goAhead(self.entryName.get())) 
        self.go.place(relx = 0.87, rely = 0.75) 
        
        self.opening = Button(self.login, text = "Show", font = "Helvetica 14 bold",command = lambda: self.show(self.entryNameURL.get())) 
        
        self.opening.place(relx = 0.87,rely = 0.48)

        rcv = threading.Thread(target=self.receive) 
        rcv.start()  
        self.textCons = Text(self.login, width = 10, height = 1, bg = "#FFFFFF", fg = "#000000", font = "Helvetica 14", padx = 1, pady = 1) 

        self.textCons.place(relheight =.9, relwidth = .5, rely = 0) 

        self.labelBottom = Label(self.login, bg = "#ABB2B9", height = 80) 

        self.labelBottom.place(relwidth = 1, rely = 0.825) 
        
        self.entryMsg = Entry(self.labelBottom, bg = "#2C3E50", fg = "#FFFFFF", font = "Helvetica 13 bold") 
        
        self.entryMsg.place(relwidth = 0.74,relheight = 0.06,rely = 0.008,relx = 0.011)

        self.entryMsg.config(state = DISABLED)
        self.buttonMsg = Button(self.labelBottom,text = "Send",font = "Helvetica 14 bold",width = 20,bg = "#000000",fg="#EEEEEE",command = lambda : self.sendButton(self.entryMsg.get())) 

        self.buttonMsg.place(relx = 0.77,rely = 0.008,relheight = 0.06,relwidth = 0.22)
        self.buttonMsg.config(state = DISABLED)
        
        self.Window.mainloop() 

    def goAhead(self,name):
        self.name=name
        if(name!=""):
            self.entryName.config(state = DISABLED)
            self.buttonMsg.config(state = NORMAL)
    def show(self,ur):
        if(ur==""):             
            fln=filedialog.askopenfilename(initialdir=os.getcwd(),title="Select Image File ",
            filetypes=(("JPG File","*.jpg"),("PNG File","*.png"),("All file","*.*")))
            self.imges=Image.open(fln)
            self.imges.thumbnail((350,300))
            self.imges=ImageTk.PhotoImage(self.imges)
        else:
            urllib.request.urlretrieve(ur,'my_image.png')
            self.imges=Image.open('my_image.png')
            self.imges.thumbnail((350,300))
            self.imges=ImageTk.PhotoImage(self.imges)

        self.image = Label(self.login,
        justify = CENTER,
        font = "Helvetica 14",
        image=self.imges) 
        self.image.place(relheight = 0.3, relwidth = 0.4, relx = 0.55, rely = 0.1) 

    def sendButton(self, msg): 
        self.entryMsg.config(state = NORMAL)
        self.textCons.config(state = DISABLED) 
        self.msg=msg 
        self.entryMsg.delete(0, END) 
        snd= threading.Thread(target = self.sendMessage) 
        snd.start() 

    def receive(self): 
        while True: 
            try: 
                message = client.recv(1024).decode(FORMAT) 
 
                if message == 'NAME': 
                    client.send(self.name.encode(FORMAT)) 
                else:  
                    self.textCons.config(state = NORMAL) 
                    self.textCons.insert(END,message+"\n\n") 
                    self.textCons.config(state = DISABLED)
                    self.textCons.see(END) 
            except: 
                print("An error occured!")
                client.close()
                break

    def sendMessage(self):
        self.textCons.config(state=DISABLED) 
        while True: 
            message = (f"{self.name}: {self.msg}") 
            client.send(message.encode(FORMAT))	 
            break

g = GUI() 